In [15]:
import numpy as np
import pandas as pd

trainData = pd.read_csv("train.csv")

In [23]:
trainX = trainData.iloc[:, 1:]
trainY = trainData.iloc[:,0]

oneHotTrainY = np.zeros((len(trainY), 10))
for i,val in enumerate(trainY):
    oneHotTrainY[i][val] = 1
    

    
oneHotTrainY

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [44]:
#convTrainX = np.zeros((42000,28,28,1))
#for i, pixels in enumerate(np.array(trainX)):
#    convTrainX[i] = np.array(pixels).reshape((28,28,1))
#print(convTrainX.shape)
trainX

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Flatten, SeparableConv2D, Dropout, MaxPooling2D
model = Sequential()
model.add(Dropout(0.5, input_shape = (28**2, )))
model.add(Dense(128, activation = 'relu'))

model.add(Dense(10, activation = 'softmax'))
model.summary()
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model.fit(trainX, oneHotTrainY, epochs = 40, validation_split = 0.2, callbacks = [EarlyStopping(patience = 5, monitor="val_accuracy")])

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_24 (Dropout)         (None, 784)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 128)               100480    
_________________________________________________________________
dense_44 (Dense)             (None, 10)                1290      
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________
Train on 33600 samples, validate on 8400 samples
Epoch 1/40
33600/33600 [==============================] - 3s 89us/step - loss: 9.5545 - accuracy: 0.6966 - val_loss: 0.6958 - val_accuracy: 0.8179
Epoch 2/40
33600/33600 [==============================] - 3s 88us/step - loss: 1.1811 - accuracy: 0.8127 - val_loss: 0.4957 - val_accuracy: 0.8717
Epoch 3/40
33600/33600 [==========

In [39]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier


def createMod(drop):
    model = Sequential()
    model.add(Dropout(drop))
    model.add(Dense(128, activation = 'relu', input_shape = ()))
    model.add(Dense(10, activation = 'softmax'))
    model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [40]:
modelCV = KerasClassifier(build_fn = createMod, epochs = 40, verbose = 1)
paramGrid = {"drop": [0.1*x for x in range(10)]}
grid = GridSearchCV(estimator=modelCV, param_grid = paramGrid, n_jobs=-1, cv=3)

In [45]:
grid_result = grid.fit(np.array(trainX), oneHotTrainY,callbacks = [EarlyStopping(patience = 5, monitor="val_accuracy")])

Epoch 1/40
42000/42000 [==============================] - 3s 65us/step - loss: 3.2089 - accuracy: 0.8683
Epoch 2/40
 3200/42000 [=>............................] - ETA: 2s - loss: 0.5731 - accuracy: 0.9181

C:\Users\noahs\anaconda3\envs\tf\lib\site-packages\keras\callbacks\callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_accuracy` which is not available. Available metrics are: loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


42000/42000 [==============================] - 3s 63us/step - loss: 0.5810 - accuracy: 0.9217
Epoch 3/40
42000/42000 [==============================] - 3s 63us/step - loss: 0.4784 - accuracy: 0.9408
Epoch 4/40
42000/42000 [==============================] - 3s 62us/step - loss: 0.4102 - accuracy: 0.9496
Epoch 5/40
42000/42000 [==============================] - 3s 63us/step - loss: 0.3673 - accuracy: 0.9547
Epoch 6/40
42000/42000 [==============================] - 3s 62us/step - loss: 0.3521 - accuracy: 0.9593
Epoch 7/40
42000/42000 [==============================] - 3s 62us/step - loss: 0.3210 - accuracy: 0.9629
Epoch 8/40
42000/42000 [==============================] - 3s 63us/step - loss: 0.2930 - accuracy: 0.9663
Epoch 9/40
42000/42000 [==============================] - 3s 63us/step - loss: 0.2885 - accuracy: 0.9682
Epoch 10/40
42000/42000 [==============================] - 3s 63us/step - loss: 0.2612 - accuracy: 0.9700
Epoch 11/40
42000/42000 [==============================] - 3s 62u

In [9]:
grid_result

GridSearchCV(cv=3,
             estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x0000027E5AFD7F48>,
             n_jobs=-1, param_grid={'convNodes': [1, 2]})

In [46]:
grid_result.cv_results_

{'mean_fit_time': array([194.44982783, 223.36912433, 228.50049384, 237.27640867,
        240.63713392, 240.0135231 , 236.55906407, 239.12900718,
        178.30828174, 163.34640598]),
 'std_fit_time': array([1.85097688, 0.67061927, 8.01538928, 1.32821815, 4.88275786,
        0.80526314, 1.67789966, 0.94409797, 0.98276476, 2.07928607]),
 'mean_score_time': array([1.69674214, 1.64052725, 1.85037486, 2.02385982, 1.82891456,
        2.11231057, 2.36470962, 1.66579986, 1.20884887, 0.46238629]),
 'std_score_time': array([0.18766544, 0.19816753, 0.39679363, 0.35774373, 0.52479075,
        0.14319067, 0.4034363 , 0.14793053, 0.29499665, 0.09218898]),
 'param_drop': masked_array(data=[0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5,
                    0.6000000000000001, 0.7000000000000001, 0.8, 0.9],
              mask=[False, False, False, False, False, False, False, False,
                    False, False],
        fill_value='?',
             dtype=object),
 'params': [{'drop': 0.0},
  {'drop'